In [ ]:
import os 
from PIL import Image
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Compose, ToTensor, Resize

def read_image(transform, path, unit):
    images = []
    labels = []
    # path = r'.\Data\seg_train'
    for num, picture_class in enumerate([os.path.join(path, file) for file in unit]):
        # print(num, picture_class)
        # num 等同于标签
        for file in os.listdir(picture_class):         # 返回列表
            image = os.path.join(picture_class, file)  # 返回完整图片路径
            images.append(transform(Image.open(image)))
            labels.append(num)
    return images, labels  

class image_dataset(Dataset):
    def __init__(self, path, unit) -> None:
        super().__init__()
        # 图片转换
        self.transform = Compose([
            Resize((128, 128)),
            ToTensor()
        ])
        self.read_image = read_image(transform=self.transform, path=path, unit=unit)
        self.images, self.labels = self.read_image

    def __len__(self,):
        return len(self.images)

    def __getitem__(self, index):
        return self.images[index], self.labels[index]

if __name__ =='__main__':
    train_data = image_dataset(path = r'.\Data\seg_train', unit = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street'])
    train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
    test_data = image_dataset(path = r'.\Data\seg_test', unit = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street'])
    test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=True)

In [ ]:
import torch.nn as nn
class Network(nn.Module):
    def __init__(self, num_classes) -> None:
        super(Network, self).__init__()
        # Alexnet网络
        self.layer_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=7, stride=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.layer_2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.layer_3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )

        self.layer_4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        
        self.layer_5 = nn.Sequential(
            nn.Linear(256*4*4, 256), 
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes),
        )
        

    def forward(self, x):
        # x: 输入图片，shape = [batch_size, in_channels, height, width]
        x = self.layer_1(x)
        x = self.layer_2(x)
        x = self.layer_3(x)
        x = self.layer_4(x)
        x = x.view(x.shape[0], -1).contiguous()
        predict = self.layer_5(x)
        return predict

if __name__ == '__main__':
    net = Network(num_classes=6)


In [ ]:
import numpy as np 
import torch
import torch.nn.functional as F
import torch.optim as optim 

def train(network, learning_rate, epoches, path, device):
    # 定义交叉熵损失函数和优化器
    train_loss = []
    loss_func = nn.CrossEntropyLoss()
    model_optim = optim.AdamW(network.parameters(), lr=learning_rate)
    # 模型训练
    for epoch in range(epoches):
        for image, label in train_loader:
            predict = network(image.to(device))
            trainlabel = F.one_hot(torch.from_numpy(np.array([int(x) for x in label])).unsqueeze(dim=0).to(torch.int64), num_classes=6).squeeze(dim=0).to(device)
            model_loss = loss_func(predict, trainlabel.float())
            # 参数优化
            model_optim.zero_grad()
            model_loss.backward()
            # 计算梯度
            model_optim.step()
            train_loss.append(model_loss.detach().cpu().item())
            acc = (predict.argmax(dim=1) == label.to(device)).float().mean()
        print('---Predict---:\n', predict.argmax(-1))
        print('---Label---:\n', label)
        print('Epoch:', epoch, 'Model Loss:', train_loss[-1], 'Accuracy:', acc)

    # 模型保存
    torch.save(network.state_dict(), path)

    import matplotlib.pyplot as plt
    plt.plot(range(len(train_loss)), train_loss)
    plt.show()
    return train_loss 
    

if __name__ =='__main__':
    train(network=net.cuda(), learning_rate=0.005, epoches=50, path=r'.\Model\alexnet.pth', device= 'cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
net = Network(num_classes=6)
net.load_state_dict(torch.load(r'.\Model\alexnet.pth'))

In [ ]:
import numpy as np

for image, label in test_loader:
    predict = net(image)
    acc = (predict.argmax(dim=1) == label).float().mean()
    print('Accuracy:', acc)
    print('---Predict---:\n', predict.argmax(-1))
    print('---Label---:\n', label)